# Preprocessing

## Tokenization

In [ ]:
from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer('basic_english')
tokens = tokenizer('I am reading a book now. I love to read books!')
print(tokens)

## Removing stopwords

In [ ]:
import nltk 

nltk.download('stopwords')

from nltk.corpus import stopwords 

stop_words = set(stopwords.words('english'))

filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
print(filtered_tokens)

## Stemming

In [ ]:
from nltk.stem import PorterStemmer 

stemmer = PorterStemmer() 

stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
print(stemmed_tokens)

## Rare word removal

In [ ]:
from nltk.probability import FreqDist 

freq_dist = FreqDist(stemmed_tokens)

threshold = 2 

common_tokens = [token for token in stemmed_tokens if freq_dist[token]>=threshold]
print(common_tokens)

## Encoding text data

### One Hot Encoding

In [ ]:
import torch 

vocab=['cat', 'dog', 'rabbit']
vocab_size = len(vocab)

one_hot_vectors = torch.eye(vocab_size)
one_hot_dict = {word: one_hot_vectors[i] for i, word in enumerate(vocab)}
print(one_hot_dict)

### Bag of Words 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer() 
corpus = ['Primer documento.', 
          'Otro documento mas, y ya van dos documentos. Cuantos documentos mas necesitamos?', 
          'Un ultimo documento para terminar la coleccion de documentos'] 

X = vectorizer.fit_transform(corpus) 
print(X.toarray())
print(vectorizer.get_feature_names_out())


### TF-IDF 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np 

vectorizer = TfidfVectorizer() 
corpus = ['Primer documento.', 
          'Otro documento mas, y ya van dos documentos. Cuantos documentos mas necesitamos?', 
          'Un ultimo documento para terminar la coleccion de documentos'] 

X = vectorizer.fit_transform(corpus) 
print(np.round(X.toarray(), 2))
print(vectorizer.get_feature_names_out())


### Embeddings

In [ ]:
with open('../data/shakespeare.txt', 'r') as file:
    raw_text = file.read()

def extract_sentences(data): 
    sentences = re.findall(r'[A-Z][^.!?]*[.!?]', data) 
    return sentences

shakespeare = extract_sentences(raw_text)

In [ ]:
# Create a list of stopwords
stop_words = set(stopwords.words("english"))

# Initialize the tokenizer and stemmer
tokenizer = get_tokenizer("basic_english")
stemmer = PorterStemmer() 

# Complete the function to preprocess sentences
def preprocess_sentences(sentences):
    processed_sentences = []
    for sentence in sentences:
        sentence = sentence.lower()
		# Tokenize the sentence
        tokens = tokenizer(sentence)
		# Remove stop words
        tokens = [token for token in tokens if token not in stop_words]        
		# Stem the tokens
        tokens = [stemmer.stem(token) for token in tokens]
        processed_sentences.append(' '.join(tokens))
    return processed_sentences

processed_shakespeare = preprocess_sentences(shakespeare)
print(processed_shakespeare[:5]) 

In [ ]:
# Define your Dataset class
class ShakespeareDataset(Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]

# Complete the encoding function
def encode_sentences(sentences):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(sentences)
    return X.toarray(), vectorizer
    
# Complete the text processing pipeline
def text_processing_pipeline(sentences):
    processed_sentences = preprocess_sentences(sentences)
    encoded_sentences, vectorizer = encode_sentences(processed_sentences)
    dataset = ShakespeareDataset(encoded_sentences)
    dataloader = DataLoader(dataset, batch_size=2, shuffle=True)
    return dataloader, vectorizer

dataloader, vectorizer = text_processing_pipeline(processed_shakespeare)


In [ ]:
# Print the vectorizer's feature names and the first 10 components of the first item
print(vectorizer.get_feature_names_out()[:500]) 
print(next(iter(dataloader))[0, :50])

## Text Classification

A text can be classified into: 
- Binary: 2 classes
- Multiclass: more than 2 classes

**Embeddings** are representations of words in multidimensional spaces, where the relations with other word representations are kept.

In [ ]:
import torch
from torch import nn 

words = ["the", "cat", "sat", "on", "the", "mat"] 
word_to_idx = {word: i for i, word in enumerate(words)}
print(word_to_idx)

In [ ]:
inputs = torch.LongTensor([word_to_idx[w] for w in words])
embedding = nn.Embedding(num_embeddings=len(words), embedding_dim=10)
output = embedding(inputs)
print(output)

# CNNs

In [ ]:
class TextClassificationCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(TextClassificationCNN, self).__init__()
        # Initialize the embedding layer 
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.conv = nn.Conv1d(embed_dim, embed_dim, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(embed_dim, 2)
    
    def forward(self, text):
        embedded = self.embedding(text).permute(0, 2, 1)
        # Pass the embedded text through the convolutional layer and apply a ReLU
        conved = F.relu(self.conv(embedded))
        conved = conved.mean(dim=2) 
        return self.fc(conved)

In [ ]:
data=[(['I', 'love', 'this', 'book'], 1),
 (['This', 'is', 'an', 'amazing', 'novel'], 1),
 (['I', 'really', 'like', 'this', 'story'], 1),
 (['I', 'do', 'not', 'like', 'this', 'book'], 0),
 (['I', 'hate', 'this', 'novel'], 0),
 (['This', 'is', 'a', 'terrible', 'story'], 0)]


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

vocab = ["I", "love", "this", "book", "do", "not", "like", "really", 'story']
word_to_idx = {word: i for i, word in enumerate(vocab)}
vocab_size = len(word_to_idx)
embed_dim = 10

model = TextClassificationCNN(10,embed_dim) 

# Define the loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(10):
    for sentence, label in data:     
        # Clear the gradients
        model.zero_grad()
        sentence = torch.LongTensor([word_to_idx.get(w, 0) for w in sentence]).unsqueeze(0) 
        label = torch.LongTensor([int(label)])
        outputs = model(sentence)
        loss = criterion(outputs, label)
        loss.backward()
        # Update the parameters
        optimizer.step()

print('Training complete!')

In [ ]:
book_reviews = [
    "I do love this story".split(),
    "I do not like this book".split()
]
for review in book_reviews:
    # Convert the review words into tensor form
    input_tensor = torch.tensor([word_to_idx[w] for w in review], dtype=torch.long).unsqueeze(0) 
    # Get the model's output
    outputs = model(input_tensor)
    # Find the index of the most likely sentiment category
    _, predicted_label = torch.max(outputs.data, 1)
    # Convert the predicted label into a sentiment string
    sentiment = "Positive" if predicted_label.item()==1 else "Negative"
    print(predicted_label.item())
    print(f"Book Review: {' '.join(review)}")
    print(f"Sentiment: {sentiment}\n")